<a href="https://colab.research.google.com/github/Imtiazul-Islam/ai-models/blob/main/TTS/Ultimate-TTS-Colab-CPU-Only-Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🚀 **CELL 1: Installation & Setup**
#@markdown ### One-click installation - Run this first!

import subprocess
import sys
import os

print("╔" + "═"*58 + "╗")
print("║" + " "*15 + "🎙️ PRO TTS STUDIO v2.0" + " "*16 + "║")
print("║" + " "*12 + "Professional Text-to-Speech System" + " "*11 + "║")
print("╚" + "═"*58 + "╝")

# ============ INSTALLATION ============
packages = [
    "edge-tts", "gTTS", "pyttsx3", "pydub", "nest-asyncio",
    "ipywidgets", "num2words", "inflect", "langdetect"
]

print("\n⏳ Installing packages...")
for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
print("✅ Packages installed!")

# System dependencies
print("⏳ Installing system dependencies...")
subprocess.run(["apt-get", "update", "-qq"], capture_output=True)
subprocess.run(["apt-get", "install", "-y", "-qq", "ffmpeg"], capture_output=True)
print("✅ FFmpeg installed!")

# ============ IMPORTS ============
print("⏳ Loading libraries...")

import json
import time
import asyncio
import hashlib
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any

import edge_tts
import nest_asyncio
from gtts import gTTS
from pydub import AudioSegment
from pydub.effects import normalize
from num2words import num2words
import inflect

import ipywidgets as widgets
from IPython.display import display, Audio, HTML, clear_output

nest_asyncio.apply()

# ============ DIRECTORIES ============
BASE_DIR = Path("/content/pro_tts_studio")
DIRS = {
    'audio': BASE_DIR / "audio",
    'projects': BASE_DIR / "projects",
    'exports': BASE_DIR / "exports",
    'temp': BASE_DIR / "temp",
    'presets': BASE_DIR / "presets",
}
for d in DIRS.values():
    d.mkdir(parents=True, exist_ok=True)

# ============ LOAD VOICES ============
print("⏳ Loading voices...")

async def _load_voices():
    return await edge_tts.list_voices()

ALL_VOICES = asyncio.get_event_loop().run_until_complete(_load_voices())

print(f"""
╔{'═'*58}╗
║{'✅ INSTALLATION COMPLETE!':^58}║
╠{'═'*58}╣
║  📊 {len(ALL_VOICES)} Premium Voices Loaded{' '*29}║
║  🎯 Ready for Professional Audio Production{' '*14}║
║  📁 Output Directory: {str(BASE_DIR)[:33]+'...' if len(str(BASE_DIR))>33 else str(BASE_DIR):<36}║
╚{'═'*58}╝

▶️ Run Cell 2 to load the TTS Engine
""")

In [ ]:
#@title 🔧 **CELL 2: Professional TTS Engine**
#@markdown ### Core engine with presets, styles, and professional features

# ══════════════════════════════════════════════════════════════════════════════
#                              CONTENT PRESETS
# ══════════════════════════════════════════════════════════════════════════════

CONTENT_PRESETS = {
    # ===== VIDEO CONTENT =====
    "youtube_explainer": {
        "name": "📺 YouTube Explainer",
        "description": "Clear, engaging narration for educational videos",
        "category": "Video",
        "voice": "en-US-DavisNeural",
        "speed": 1.05,
        "pitch": 1.0,
        "style": "friendly",
        "settings": {
            "stability": 0.75,
            "clarity": 0.85,
            "pause_strength": "medium"
        }
    },
    "youtube_shorts": {
        "name": "📱 YouTube Shorts / Reels",
        "description": "Fast-paced, energetic for short-form content",
        "category": "Video",
        "voice": "en-US-JennyNeural",
        "speed": 1.15,
        "pitch": 1.05,
        "style": "excited",
        "settings": {
            "stability": 0.65,
            "clarity": 0.9,
            "pause_strength": "weak"
        }
    },
    "documentary": {
        "name": "🎬 Documentary",
        "description": "Deep, authoritative voice for documentaries",
        "category": "Video",
        "voice": "en-US-GuyNeural",
        "speed": 0.95,
        "pitch": 0.95,
        "style": "serious",
        "settings": {
            "stability": 0.85,
            "clarity": 0.8,
            "pause_strength": "strong"
        }
    },
    "tutorial": {
        "name": "🎓 Tutorial / How-To",
        "description": "Patient, clear instruction style",
        "category": "Video",
        "voice": "en-US-AriaNeural",
        "speed": 0.9,
        "pitch": 1.0,
        "style": "instructional",
        "settings": {
            "stability": 0.8,
            "clarity": 0.9,
            "pause_strength": "strong"
        }
    },

    # ===== AUDIO CONTENT =====
    "podcast": {
        "name": "🎙️ Podcast",
        "description": "Conversational, warm tone for podcasts",
        "category": "Audio",
        "voice": "en-US-JennyNeural",
        "speed": 1.0,
        "pitch": 1.0,
        "style": "conversational",
        "settings": {
            "stability": 0.7,
            "clarity": 0.75,
            "pause_strength": "medium"
        }
    },
    "audiobook_fiction": {
        "name": "📖 Audiobook (Fiction)",
        "description": "Expressive storytelling for fiction",
        "category": "Audio",
        "voice": "en-GB-SoniaNeural",
        "speed": 0.9,
        "pitch": 1.0,
        "style": "storytelling",
        "settings": {
            "stability": 0.65,
            "clarity": 0.8,
            "pause_strength": "strong"
        }
    },
    "audiobook_nonfiction": {
        "name": "📚 Audiobook (Non-Fiction)",
        "description": "Clear, informative for non-fiction",
        "category": "Audio",
        "voice": "en-US-GuyNeural",
        "speed": 0.95,
        "pitch": 1.0,
        "style": "informative",
        "settings": {
            "stability": 0.8,
            "clarity": 0.85,
            "pause_strength": "medium"
        }
    },
    "meditation": {
        "name": "🧘 Meditation / Relaxation",
        "description": "Calm, soothing voice for wellness content",
        "category": "Audio",
        "voice": "en-US-AriaNeural",
        "speed": 0.8,
        "pitch": 0.95,
        "style": "calm",
        "settings": {
            "stability": 0.9,
            "clarity": 0.7,
            "pause_strength": "strong"
        }
    },
    "asmr": {
        "name": "🌙 ASMR Style",
        "description": "Soft, whisper-like quality",
        "category": "Audio",
        "voice": "en-US-JennyNeural",
        "speed": 0.75,
        "pitch": 0.9,
        "style": "soft",
        "settings": {
            "stability": 0.85,
            "clarity": 0.6,
            "pause_strength": "strong"
        }
    },

    # ===== COMMERCIAL =====
    "commercial_tv": {
        "name": "📺 TV Commercial",
        "description": "Professional, persuasive advertising voice",
        "category": "Commercial",
        "voice": "en-US-DavisNeural",
        "speed": 1.05,
        "pitch": 1.0,
        "style": "professional",
        "settings": {
            "stability": 0.8,
            "clarity": 0.9,
            "pause_strength": "weak"
        }
    },
    "radio_ad": {
        "name": "📻 Radio Advertisement",
        "description": "Energetic, attention-grabbing",
        "category": "Commercial",
        "voice": "en-US-GuyNeural",
        "speed": 1.1,
        "pitch": 1.05,
        "style": "energetic",
        "settings": {
            "stability": 0.7,
            "clarity": 0.85,
            "pause_strength": "weak"
        }
    },
    "promo": {
        "name": "🎉 Promotional / Trailer",
        "description": "Epic, dramatic announcement style",
        "category": "Commercial",
        "voice": "en-US-GuyNeural",
        "speed": 0.95,
        "pitch": 0.9,
        "style": "dramatic",
        "settings": {
            "stability": 0.75,
            "clarity": 0.85,
            "pause_strength": "medium"
        }
    },

    # ===== BUSINESS =====
    "corporate": {
        "name": "🏢 Corporate / Business",
        "description": "Professional, trustworthy business tone",
        "category": "Business",
        "voice": "en-US-AriaNeural",
        "speed": 1.0,
        "pitch": 1.0,
        "style": "professional",
        "settings": {
            "stability": 0.85,
            "clarity": 0.85,
            "pause_strength": "medium"
        }
    },
    "elearning": {
        "name": "🎓 E-Learning / Training",
        "description": "Clear, patient educational delivery",
        "category": "Business",
        "voice": "en-US-JennyNeural",
        "speed": 0.95,
        "pitch": 1.0,
        "style": "instructional",
        "settings": {
            "stability": 0.8,
            "clarity": 0.9,
            "pause_strength": "strong"
        }
    },
    "presentation": {
        "name": "📊 Presentation",
        "description": "Confident, engaging presentation style",
        "category": "Business",
        "voice": "en-US-DavisNeural",
        "speed": 1.0,
        "pitch": 1.0,
        "style": "confident",
        "settings": {
            "stability": 0.8,
            "clarity": 0.85,
            "pause_strength": "medium"
        }
    },
    "ivr_phone": {
        "name": "📞 IVR / Phone System",
        "description": "Clear, professional phone prompts",
        "category": "Business",
        "voice": "en-US-AriaNeural",
        "speed": 0.95,
        "pitch": 1.0,
        "style": "neutral",
        "settings": {
            "stability": 0.9,
            "clarity": 0.95,
            "pause_strength": "strong"
        }
    },

    # ===== NEWS & ANNOUNCEMENTS =====
    "news_anchor": {
        "name": "📰 News Anchor",
        "description": "Authoritative news delivery",
        "category": "News",
        "voice": "en-US-GuyNeural",
        "speed": 1.05,
        "pitch": 1.0,
        "style": "authoritative",
        "settings": {
            "stability": 0.85,
            "clarity": 0.9,
            "pause_strength": "medium"
        }
    },
    "sports_commentary": {
        "name": "⚽ Sports Commentary",
        "description": "Energetic sports announcing",
        "category": "News",
        "voice": "en-US-DavisNeural",
        "speed": 1.15,
        "pitch": 1.1,
        "style": "excited",
        "settings": {
            "stability": 0.6,
            "clarity": 0.8,
            "pause_strength": "weak"
        }
    },

    # ===== CHARACTERS =====
    "character_friendly": {
        "name": "😊 Friendly Character",
        "description": "Warm, approachable character voice",
        "category": "Character",
        "voice": "en-US-JennyNeural",
        "speed": 1.0,
        "pitch": 1.1,
        "style": "friendly",
        "settings": {
            "stability": 0.6,
            "clarity": 0.75,
            "pause_strength": "medium"
        }
    },
    "character_serious": {
        "name": "😐 Serious Character",
        "description": "Stern, authoritative character",
        "category": "Character",
        "voice": "en-US-GuyNeural",
        "speed": 0.95,
        "pitch": 0.9,
        "style": "serious",
        "settings": {
            "stability": 0.8,
            "clarity": 0.85,
            "pause_strength": "strong"
        }
    },
    "gaming_narrator": {
        "name": "🎮 Gaming Narrator",
        "description": "Epic game narration style",
        "category": "Character",
        "voice": "en-GB-RyanNeural",
        "speed": 0.95,
        "pitch": 0.95,
        "style": "dramatic",
        "settings": {
            "stability": 0.7,
            "clarity": 0.85,
            "pause_strength": "medium"
        }
    },
}

# ══════════════════════════════════════════════════════════════════════════════
#                              VOICE DATABASE
# ══════════════════════════════════════════════════════════════════════════════

# Curated voice categories for easy selection
VOICE_CATEGORIES = {
    "recommended": {
        "name": "⭐ Recommended",
        "voices": {
            "en-US-JennyNeural": {"name": "Jenny", "gender": "F", "style": "Versatile, warm, natural", "best_for": "YouTube, Podcast, General"},
            "en-US-GuyNeural": {"name": "Guy", "gender": "M", "style": "Confident, professional", "best_for": "Documentary, Business, News"},
            "en-US-AriaNeural": {"name": "Aria", "gender": "F", "style": "Clear, friendly, professional", "best_for": "E-Learning, Corporate"},
            "en-US-DavisNeural": {"name": "Davis", "gender": "M", "style": "Engaging, trustworthy", "best_for": "Explainers, Tutorials"},
            "en-GB-SoniaNeural": {"name": "Sonia", "gender": "F", "style": "Elegant, expressive", "best_for": "Audiobooks, Storytelling"},
            "en-GB-RyanNeural": {"name": "Ryan", "gender": "M", "style": "Deep, authoritative", "best_for": "Documentary, Gaming"},
        }
    },
    "narration": {
        "name": "📖 Narration",
        "voices": {
            "en-US-GuyNeural": {"name": "Guy", "gender": "M", "style": "Professional narrator"},
            "en-GB-SoniaNeural": {"name": "Sonia", "gender": "F", "style": "British narrator"},
            "en-AU-WilliamNeural": {"name": "William", "gender": "M", "style": "Australian narrator"},
            "en-US-JennyNeural": {"name": "Jenny", "gender": "F", "style": "American narrator"},
        }
    },
    "conversational": {
        "name": "💬 Conversational",
        "voices": {
            "en-US-JennyNeural": {"name": "Jenny", "gender": "F", "style": "Natural, friendly"},
            "en-US-DavisNeural": {"name": "Davis", "gender": "M", "style": "Warm, approachable"},
            "en-US-AriaNeural": {"name": "Aria", "gender": "F", "style": "Bright, engaging"},
            "en-US-TonyNeural": {"name": "Tony", "gender": "M", "style": "Casual, relatable"},
        }
    },
    "professional": {
        "name": "💼 Professional",
        "voices": {
            "en-US-AriaNeural": {"name": "Aria", "gender": "F", "style": "Corporate, clear"},
            "en-US-GuyNeural": {"name": "Guy", "gender": "M", "style": "Business professional"},
            "en-GB-RyanNeural": {"name": "Ryan", "gender": "M", "style": "British professional"},
            "en-AU-NatashaNeural": {"name": "Natasha", "gender": "F", "style": "Australian professional"},
        }
    },
    "energetic": {
        "name": "⚡ Energetic",
        "voices": {
            "en-US-JennyNeural": {"name": "Jenny", "gender": "F", "style": "Upbeat, dynamic"},
            "en-US-DavisNeural": {"name": "Davis", "gender": "M", "style": "Enthusiastic"},
            "en-US-SaraNeural": {"name": "Sara", "gender": "F", "style": "Bright, cheerful"},
        }
    },
    "calm": {
        "name": "🧘 Calm & Soothing",
        "voices": {
            "en-US-AriaNeural": {"name": "Aria", "gender": "F", "style": "Gentle, calming"},
            "en-GB-SoniaNeural": {"name": "Sonia", "gender": "F", "style": "Soothing, elegant"},
            "en-US-JennyNeural": {"name": "Jenny", "gender": "F", "style": "Warm, relaxing"},
        }
    },
}

# Build complete voice lookup
VOICES_LOOKUP = {}
for v in ALL_VOICES:
    VOICES_LOOKUP[v['ShortName']] = {
        'locale': v['Locale'],
        'gender': v['Gender'],
        'name': v['ShortName'].split('-')[-1].replace('Neural', ''),
    }

# Group by locale
VOICES_BY_LOCALE = {}
for v in ALL_VOICES:
    locale = v['Locale']
    if locale not in VOICES_BY_LOCALE:
        VOICES_BY_LOCALE[locale] = []
    VOICES_BY_LOCALE[locale].append(v)

# ══════════════════════════════════════════════════════════════════════════════
#                              TEXT PROCESSOR
# ══════════════════════════════════════════════════════════════════════════════

class TextProcessor:
    """Intelligent text preprocessing for natural speech"""

    def __init__(self):
        self.inflect_engine = inflect.engine()

        self.abbreviations = {
            r'\bmr\.': 'mister', r'\bmrs\.': 'missus', r'\bdr\.': 'doctor',
            r'\bprof\.': 'professor', r'\bvs\.': 'versus', r'\betc\.': 'etcetera',
            r'\be\.g\.': 'for example', r'\bi\.e\.': 'that is',
            r'\bst\.': 'street', r'\bave\.': 'avenue', r'\bblvd\.': 'boulevard',
            r'\bjr\.': 'junior', r'\bsr\.': 'senior',
        }

        self.symbols = {
            '&': ' and ', '@': ' at ', '#': ' number ',
            '%': ' percent ', '+': ' plus ', '=': ' equals ',
            '$': ' dollars ', '€': ' euros ', '£': ' pounds ',
        }

    def expand_numbers(self, text):
        """Convert numbers to words"""
        import re
        def replace(match):
            try:
                num = match.group()
                if '.' in num:
                    parts = num.split('.')
                    return num2words(int(parts[0])) + ' point ' + ' '.join(num2words(int(d)) for d in parts[1])
                return num2words(int(num))
            except:
                return match.group()
        return re.sub(r'\b\d+\.?\d*\b', replace, text)

    def expand_abbreviations(self, text):
        """Expand common abbreviations"""
        import re
        for pattern, replacement in self.abbreviations.items():
            text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
        return text

    def replace_symbols(self, text):
        """Replace symbols with words"""
        for symbol, word in self.symbols.items():
            text = text.replace(symbol, word)
        return text

    def add_pauses(self, text, strength="medium"):
        """Add natural pauses based on punctuation"""
        import re
        pause_map = {
            "weak": {"...": ".. ", ",": ", ", ";": "; "},
            "medium": {"...": "... ", ",": ",, ", ";": ";; "},
            "strong": {"...": ".... ", ",": ",,, ", ";": ";;; "},
        }
        pauses = pause_map.get(strength, pause_map["medium"])

        for char, replacement in pauses.items():
            text = text.replace(char, replacement)
        return text

    def clean(self, text):
        """Clean and normalize text"""
        import re
        text = ' '.join(text.split())  # Normalize whitespace
        text = re.sub(r'([.!?])\1+', r'\1', text)  # Remove repeated punctuation
        text = re.sub(r'([.!?,])([A-Za-z])', r'\1 \2', text)  # Add space after punctuation
        return text.strip()

    def process(self, text, options=None):
        """Full preprocessing pipeline"""
        options = options or {}

        if options.get('expand_abbreviations', True):
            text = self.expand_abbreviations(text)
        if options.get('expand_numbers', True):
            text = self.expand_numbers(text)
        if options.get('replace_symbols', True):
            text = self.replace_symbols(text)
        if options.get('add_pauses', False):
            text = self.add_pauses(text, options.get('pause_strength', 'medium'))

        text = self.clean(text)
        return text


# ══════════════════════════════════════════════════════════════════════════════
#                              AUDIO PROCESSOR
# ══════════════════════════════════════════════════════════════════════════════

class AudioProcessor:
    """Professional audio processing"""

    def load(self, path):
        return AudioSegment.from_file(path)

    def save(self, audio, path, format='mp3', bitrate='192k'):
        audio.export(path, format=format, bitrate=bitrate)
        return path

    def normalize(self, audio, headroom=0.1):
        return normalize(audio, headroom)

    def change_speed(self, audio, speed):
        if speed == 1.0:
            return audio
        new_rate = int(audio.frame_rate * speed)
        return audio._spawn(audio.raw_data, overrides={'frame_rate': new_rate}).set_frame_rate(audio.frame_rate)

    def fade(self, audio, fade_in_ms=0, fade_out_ms=0):
        if fade_in_ms > 0:
            audio = audio.fade_in(fade_in_ms)
        if fade_out_ms > 0:
            audio = audio.fade_out(fade_out_ms)
        return audio

    def add_silence(self, audio, start_ms=0, end_ms=0):
        if start_ms > 0:
            audio = AudioSegment.silent(duration=start_ms) + audio
        if end_ms > 0:
            audio = audio + AudioSegment.silent(duration=end_ms)
        return audio

    def trim_silence(self, audio, threshold=-50, chunk_size=10):
        start = 0
        for i in range(0, len(audio), chunk_size):
            if audio[i:i+chunk_size].dBFS > threshold:
                start = i
                break
        end = len(audio)
        for i in range(len(audio), 0, -chunk_size):
            if audio[i-chunk_size:i].dBFS > threshold:
                end = i
                break
        return audio[start:end]

    def concatenate(self, segments, gap_ms=500):
        if not segments:
            return AudioSegment.empty()
        result = segments[0]
        for seg in segments[1:]:
            result += AudioSegment.silent(duration=gap_ms) + seg
        return result

    def get_duration(self, audio):
        return len(audio) / 1000.0


# ══════════════════════════════════════════════════════════════════════════════
#                              MAIN TTS ENGINE
# ══════════════════════════════════════════════════════════════════════════════

class ProTTSEngine:
    """Professional TTS Engine with presets and advanced features"""

    def __init__(self):
        self.text_processor = TextProcessor()
        self.audio_processor = AudioProcessor()
        self.presets = CONTENT_PRESETS
        self.voice_categories = VOICE_CATEGORIES
        self.history = []

    def get_preset(self, preset_key):
        """Get a content preset"""
        return self.presets.get(preset_key, self.presets['youtube_explainer'])

    def list_presets(self, category=None):
        """List available presets"""
        if category:
            return {k: v for k, v in self.presets.items() if v['category'] == category}
        return self.presets

    def get_voice_info(self, voice_id):
        """Get voice information"""
        return VOICES_LOOKUP.get(voice_id, {})

    # ========== EDGE TTS ==========
    async def _generate_edge(self, text, voice, rate, pitch, volume, output_path):
        """Generate with Edge TTS"""
        rate_str = f"{'+' if rate >= 1 else ''}{int((rate-1)*100)}%"
        pitch_str = f"{'+' if pitch >= 1 else ''}{int((pitch-1)*50)}Hz"
        vol_str = f"{'+' if volume >= 1 else ''}{int((volume-1)*100)}%"

        communicate = edge_tts.Communicate(text, voice, rate=rate_str, pitch=pitch_str, volume=vol_str)
        await communicate.save(output_path)
        return output_path

    # ========== GOOGLE TTS ==========
    def _generate_google(self, text, language, slow, output_path):
        """Generate with Google TTS"""
        tts = gTTS(text=text, lang=language, slow=slow)
        tts.save(output_path)
        return output_path

    # ========== MAIN SYNTHESIS ==========
    def synthesize(
        self,
        text: str,
        preset: str = None,
        voice: str = None,
        speed: float = 1.0,
        pitch: float = 1.0,
        volume: float = 1.0,
        engine: str = 'edge',
        language: str = 'en',
        preprocess: bool = True,
        effects: dict = None,
        output_path: str = None
    ) -> dict:
        """
        Main synthesis function

        Args:
            text: Text to synthesize
            preset: Content preset key (e.g., 'youtube_explainer')
            voice: Voice ID (overrides preset voice)
            speed: Speech speed 0.5-2.0
            pitch: Pitch adjustment 0.5-1.5
            volume: Volume 0.0-2.0
            engine: 'edge' or 'google'
            language: Language code
            preprocess: Apply text preprocessing
            effects: Audio effects dict
            output_path: Custom output path

        Returns:
            dict with path, duration, settings
        """
        start_time = time.time()

        # Apply preset if specified
        if preset and preset in self.presets:
            p = self.presets[preset]
            voice = voice or p['voice']
            speed = p['speed']
            pitch = p['pitch']

        # Default voice
        voice = voice or 'en-US-JennyNeural'

        # Preprocess text
        processed_text = self.text_processor.process(text) if preprocess else text

        # Generate output path
        if not output_path:
            timestamp = int(time.time() * 1000)
            output_path = str(DIRS['audio'] / f"tts_{timestamp}.mp3")

        # Generate audio
        if engine == 'edge':
            asyncio.get_event_loop().run_until_complete(
                self._generate_edge(processed_text, voice, speed, pitch, volume, output_path)
            )
        elif engine == 'google':
            lang_code = language.split('-')[0] if '-' in language else language
            self._generate_google(processed_text, lang_code, speed < 0.8, output_path)

        # Apply effects
        if effects:
            audio = self.audio_processor.load(output_path)

            if effects.get('normalize'):
                audio = self.audio_processor.normalize(audio)
            if effects.get('trim_silence'):
                audio = self.audio_processor.trim_silence(audio)
            if effects.get('fade_in'):
                audio = self.audio_processor.fade(audio, fade_in_ms=effects['fade_in'])
            if effects.get('fade_out'):
                audio = self.audio_processor.fade(audio, fade_out_ms=effects['fade_out'])
            if effects.get('silence_start'):
                audio = self.audio_processor.add_silence(audio, start_ms=effects['silence_start'])
            if effects.get('silence_end'):
                audio = self.audio_processor.add_silence(audio, end_ms=effects['silence_end'])

            self.audio_processor.save(audio, output_path)

        # Get duration
        audio = self.audio_processor.load(output_path)
        duration = self.audio_processor.get_duration(audio)

        # Result
        result = {
            'path': output_path,
            'duration': round(duration, 2),
            'voice': voice,
            'speed': speed,
            'pitch': pitch,
            'preset': preset,
            'text_length': len(text),
            'generation_time': round(time.time() - start_time, 2)
        }

        # Save to history
        self.history.append({
            **result,
            'text': text[:200],
            'timestamp': datetime.now().isoformat()
        })

        return result

    # ========== MULTI-VOICE SYNTHESIS ==========
    def synthesize_multivoice(self, segments: list, gap_ms: int = 500, output_path: str = None) -> dict:
        """
        Synthesize with multiple voices

        Args:
            segments: List of dicts with 'voice', 'text', optional 'speed', 'pitch'
            gap_ms: Gap between segments
            output_path: Output path

        Returns:
            dict with path and info
        """
        audio_segments = []

        for i, seg in enumerate(segments):
            temp_path = str(DIRS['temp'] / f"seg_{i}_{int(time.time()*1000)}.mp3")

            self.synthesize(
                text=seg.get('text', ''),
                voice=seg.get('voice', 'en-US-JennyNeural'),
                speed=seg.get('speed', 1.0),
                pitch=seg.get('pitch', 1.0),
                output_path=temp_path
            )

            audio_segments.append(self.audio_processor.load(temp_path))

        # Concatenate
        final_audio = self.audio_processor.concatenate(audio_segments, gap_ms)

        if not output_path:
            output_path = str(DIRS['audio'] / f"multivoice_{int(time.time())}.mp3")

        self.audio_processor.save(final_audio, output_path)

        return {
            'path': output_path,
            'segments': len(segments),
            'duration': self.audio_processor.get_duration(final_audio)
        }

    # ========== BATCH SYNTHESIS ==========
    def synthesize_batch(self, texts: list, progress_callback=None, **kwargs) -> list:
        """Batch synthesis"""
        results = []
        total = len(texts)

        for i, text in enumerate(texts):
            result = self.synthesize(text, **kwargs)
            results.append(result)

            if progress_callback:
                progress_callback(i + 1, total)

        return results

    # ========== AUDIOBOOK ==========
    def create_audiobook(
        self,
        chapters: dict,
        title: str = "Audiobook",
        preset: str = "audiobook_fiction",
        progress_callback=None
    ) -> dict:
        """
        Create audiobook from chapters

        Args:
            chapters: Dict of {chapter_name: chapter_text}
            title: Audiobook title
            preset: Preset to use
            progress_callback: Progress callback

        Returns:
            dict with chapter files and full file
        """
        book_dir = DIRS['projects'] / title.replace(' ', '_')
        book_dir.mkdir(exist_ok=True)

        chapter_files = []
        p = self.get_preset(preset)

        for i, (name, text) in enumerate(chapters.items()):
            chapter_file = str(book_dir / f"{i+1:03d}_{name[:30].replace(' ', '_')}.mp3")

            # Add chapter header
            full_text = f"Chapter {i+1}. {name}. {text}"

            self.synthesize(
                text=full_text,
                voice=p['voice'],
                speed=p['speed'],
                pitch=p['pitch'],
                output_path=chapter_file
            )

            chapter_files.append(chapter_file)

            if progress_callback:
                progress_callback(i + 1, len(chapters), name)

        # Create full audiobook
        all_audio = [self.audio_processor.load(f) for f in chapter_files]
        full_audio = self.audio_processor.concatenate(all_audio, 2000)

        full_path = str(book_dir / f"{title.replace(' ', '_')}_complete.mp3")
        self.audio_processor.save(full_audio, full_path)

        return {
            'chapters': chapter_files,
            'full': full_path,
            'duration': self.audio_processor.get_duration(full_audio)
        }


# ============ INITIALIZE ENGINE ============
tts = ProTTSEngine()

print(f"""
╔{'═'*58}╗
║{'🔧 PRO TTS ENGINE LOADED':^58}║
╠{'═'*58}╣
║                                                          ║
║  📦 Content Presets: {len(CONTENT_PRESETS):<35}║
║  🎤 Voice Categories: {len(VOICE_CATEGORIES):<34}║
║  🌍 Total Voices: {len(ALL_VOICES):<38}║
║                                                          ║
║  ✨ Features:                                            ║
║     • Professional Content Presets                       ║
║     • Multi-Voice Synthesis                              ║
║     • Audiobook Creation                                 ║
║     • Advanced Audio Processing                          ║
║     • Batch Generation                                   ║
║                                                          ║
╚{'═'*58}╝

▶️ Run Cell 3 to launch the interface
""")

In [ ]:
#@title 🎛️ **CELL 3: Professional TTS Interface**
#@markdown ### ElevenLabs-style professional interface

# ══════════════════════════════════════════════════════════════════════════════
#                              STYLES
# ══════════════════════════════════════════════════════════════════════════════

STYLES = """
<style>
.pro-container { max-width: 1000px; margin: 0 auto; font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif; }
.pro-header { background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%); color: white; padding: 25px; border-radius: 16px; margin-bottom: 20px; text-align: center; }
.pro-header h1 { margin: 0; font-size: 28px; font-weight: 600; }
.pro-header p { margin: 8px 0 0; opacity: 0.8; font-size: 14px; }
.section-card { background: #ffffff; border: 1px solid #e0e0e0; border-radius: 12px; padding: 20px; margin-bottom: 16px; box-shadow: 0 2px 8px rgba(0,0,0,0.04); }
.section-title { font-size: 16px; font-weight: 600; color: #333; margin-bottom: 12px; display: flex; align-items: center; gap: 8px; }
.preset-grid { display: grid; grid-template-columns: repeat(auto-fill, minmax(180px, 1fr)); gap: 10px; }
.preset-card { border: 2px solid #e0e0e0; border-radius: 8px; padding: 12px; cursor: pointer; transition: all 0.2s; }
.preset-card:hover { border-color: #667eea; background: #f8f9ff; }
.preset-card.selected { border-color: #667eea; background: #eef1ff; }
.preset-name { font-weight: 600; font-size: 14px; }
.preset-desc { font-size: 11px; color: #666; margin-top: 4px; }
.voice-chip { display: inline-flex; align-items: center; gap: 6px; padding: 8px 14px; border: 2px solid #e0e0e0; border-radius: 20px; cursor: pointer; transition: all 0.2s; margin: 4px; }
.voice-chip:hover { border-color: #667eea; }
.voice-chip.selected { border-color: #667eea; background: #667eea; color: white; }
.generate-btn { background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border: none; padding: 16px 32px; border-radius: 12px; font-size: 16px; font-weight: 600; cursor: pointer; width: 100%; transition: transform 0.2s, box-shadow 0.2s; }
.generate-btn:hover { transform: translateY(-2px); box-shadow: 0 8px 20px rgba(102, 126, 234, 0.4); }
.status-bar { padding: 12px 16px; border-radius: 8px; margin-top: 12px; font-size: 14px; }
.status-ready { background: #e8f5e9; color: #2e7d32; }
.status-processing { background: #e3f2fd; color: #1565c0; }
.status-success { background: #e8f5e9; color: #2e7d32; }
.status-error { background: #ffebee; color: #c62828; }
</style>
"""

# ══════════════════════════════════════════════════════════════════════════════
#                              UI BUILDER
# ══════════════════════════════════════════════════════════════════════════════

def build_ui():
    # Header
    header = widgets.HTML(f"""
    {STYLES}
    <div class="pro-container">
        <div class="pro-header">
            <h1>🎙️ Pro TTS Studio</h1>
            <p>Professional Text-to-Speech • {len(ALL_VOICES)} Voices • {len(CONTENT_PRESETS)} Presets</p>
        </div>
    </div>
    """)

    # ========== CONTENT PRESET SECTION ==========
    preset_label = widgets.HTML('<div class="section-title">📦 Content Preset</div>')

    preset_categories = ['All', 'Video', 'Audio', 'Commercial', 'Business', 'News', 'Character']
    preset_category_dropdown = widgets.Dropdown(
        options=preset_categories,
        value='All',
        description='Category:',
        layout=widgets.Layout(width='200px')
    )

    preset_dropdown = widgets.Dropdown(
        options=[(v['name'], k) for k, v in CONTENT_PRESETS.items()],
        value='youtube_explainer',
        description='Preset:',
        layout=widgets.Layout(width='100%')
    )

    preset_info = widgets.HTML('')

    def update_preset_info(change):
        key = change['new'] if isinstance(change, dict) else preset_dropdown.value
        if key in CONTENT_PRESETS:
            p = CONTENT_PRESETS[key]
            preset_info.value = f"""
            <div style="background: #f5f7ff; padding: 12px; border-radius: 8px; margin-top: 8px;">
                <b>{p['name']}</b><br>
                <span style="color: #666; font-size: 13px;">{p['description']}</span><br>
                <span style="color: #888; font-size: 12px;">
                    Voice: {p['voice']} | Speed: {p['speed']}x | Pitch: {p['pitch']}
                </span>
            </div>
            """

    def filter_presets(change):
        cat = change['new']
        if cat == 'All':
            options = [(v['name'], k) for k, v in CONTENT_PRESETS.items()]
        else:
            options = [(v['name'], k) for k, v in CONTENT_PRESETS.items() if v['category'] == cat]
        preset_dropdown.options = options
        if options:
            preset_dropdown.value = options[0][1]
            update_preset_info(None)

    preset_dropdown.observe(update_preset_info, names='value')
    preset_category_dropdown.observe(filter_presets, names='value')
    update_preset_info(None)

    # ========== VOICE SECTION ==========
    voice_label = widgets.HTML('<div class="section-title">🎤 Voice Selection</div>')

    voice_mode = widgets.ToggleButtons(
        options=['Use Preset Voice', 'Custom Voice'],
        value='Use Preset Voice',
        layout=widgets.Layout(margin='0 0 12px 0')
    )

    voice_category = widgets.Dropdown(
        options=[(v['name'], k) for k, v in VOICE_CATEGORIES.items()],
        value='recommended',
        description='Category:',
        layout=widgets.Layout(width='250px')
    )

    voice_select = widgets.Dropdown(
        options=[],
        description='Voice:',
        layout=widgets.Layout(width='100%')
    )

    voice_container = widgets.VBox([voice_category, voice_select])
    voice_container.layout.display = 'none'

    def update_voice_options(change):
        cat_key = voice_category.value
        if cat_key in VOICE_CATEGORIES:
            voices = VOICE_CATEGORIES[cat_key]['voices']
            options = []
            for vid, vinfo in voices.items():
                emoji = "👩" if vinfo.get('gender') == 'F' else "👨"
                label = f"{emoji} {vinfo['name']} - {vinfo.get('style', '')}"
                options.append((label, vid))
            voice_select.options = options
            if options:
                voice_select.value = options[0][1]

    def toggle_voice_mode(change):
        if change['new'] == 'Custom Voice':
            voice_container.layout.display = 'block'
            update_voice_options(None)
        else:
            voice_container.layout.display = 'none'

    voice_mode.observe(toggle_voice_mode, names='value')
    voice_category.observe(update_voice_options, names='value')

    # ========== TEXT INPUT ==========
    text_label = widgets.HTML('<div class="section-title">📝 Text Input</div>')

    text_input = widgets.Textarea(
        value="Welcome to Pro TTS Studio! This is a professional text-to-speech system with high-quality voices and intelligent presets. Try selecting different presets to hear how they sound.",
        placeholder='Enter your text here...',
        layout=widgets.Layout(width='100%', height='150px')
    )

    char_count = widgets.HTML('<span style="color: #888; font-size: 12px;">Characters: 0</span>')

    def update_char_count(change):
        count = len(change['new'])
        color = '#c62828' if count > 5000 else '#888'
        char_count.value = f'<span style="color: {color}; font-size: 12px;">Characters: {count}</span>'

    text_input.observe(update_char_count, names='value')
    update_char_count({'new': text_input.value})

    # ========== SETTINGS ==========
    settings_label = widgets.HTML('<div class="section-title">⚙️ Fine-Tune Settings</div>')

    use_custom_settings = widgets.Checkbox(
        value=False,
        description='Override preset settings',
        layout=widgets.Layout(width='auto')
    )

    speed_slider = widgets.FloatSlider(
        value=1.0, min=0.5, max=2.0, step=0.05,
        description='Speed:',
        readout_format='.2f',
        layout=widgets.Layout(width='100%')
    )

    pitch_slider = widgets.FloatSlider(
        value=1.0, min=0.5, max=1.5, step=0.05,
        description='Pitch:',
        readout_format='.2f',
        layout=widgets.Layout(width='100%')
    )

    settings_container = widgets.VBox([speed_slider, pitch_slider])
    settings_container.layout.display = 'none'

    def toggle_settings(change):
        settings_container.layout.display = 'block' if change['new'] else 'none'

    use_custom_settings.observe(toggle_settings, names='value')

    # ========== AUDIO EFFECTS ==========
    effects_label = widgets.HTML('<div class="section-title">🎚️ Audio Effects</div>')

    effect_normalize = widgets.Checkbox(value=True, description='🔊 Normalize volume')
    effect_trim = widgets.Checkbox(value=False, description='✂️ Trim silence')
    effect_fade = widgets.Checkbox(value=False, description='🌊 Fade in/out')

    # ========== GENERATE BUTTON ==========
    generate_btn = widgets.Button(
        description='🎙️ Generate Speech',
        button_style='success',
        layout=widgets.Layout(width='100%', height='55px')
    )
    generate_btn.style.font_weight = 'bold'

    preview_btn = widgets.Button(
        description='👁️ Preview (first 50 words)',
        button_style='info',
        layout=widgets.Layout(width='100%', height='40px')
    )

    # ========== OUTPUT ==========
    status = widgets.HTML('<div class="status-bar status-ready">✅ Ready to generate</div>')
    output = widgets.Output()

    # ========== GENERATION LOGIC ==========
    def generate(preview=False):
        with output:
            clear_output()

            text = text_input.value.strip()
            if not text:
                status.value = '<div class="status-bar status-error">❌ Please enter some text</div>'
                return

            if preview:
                words = text.split()[:50]
                text = ' '.join(words) + ('...' if len(text.split()) > 50 else '')

            status.value = '<div class="status-bar status-processing">⏳ Generating speech...</div>'

            try:
                # Get preset
                preset_key = preset_dropdown.value
                preset = CONTENT_PRESETS.get(preset_key)

                # Determine voice
                if voice_mode.value == 'Custom Voice':
                    voice = voice_select.value
                else:
                    voice = preset['voice'] if preset else 'en-US-JennyNeural'

                # Get settings
                if use_custom_settings.value:
                    speed = speed_slider.value
                    pitch = pitch_slider.value
                else:
                    speed = preset['speed'] if preset else 1.0
                    pitch = preset['pitch'] if preset else 1.0

                # Effects
                effects = {}
                if effect_normalize.value:
                    effects['normalize'] = True
                if effect_trim.value:
                    effects['trim_silence'] = True
                if effect_fade.value:
                    effects['fade_in'] = 300
                    effects['fade_out'] = 300

                # Generate
                result = tts.synthesize(
                    text=text,
                    voice=voice,
                    speed=speed,
                    pitch=pitch,
                    effects=effects if effects else None
                )

                # Display result
                status.value = f'''
                <div class="status-bar status-success">
                    ✅ Generated successfully! Duration: {result["duration"]}s | Voice: {voice}
                </div>
                '''

                display(Audio(result['path'], autoplay=True))

                # Download link
                print(f"\n📁 Saved to: {result['path']}")
                print(f"⏱️ Generation time: {result['generation_time']}s")

            except Exception as e:
                status.value = f'<div class="status-bar status-error">❌ Error: {str(e)}</div>'
                import traceback
                traceback.print_exc()

    generate_btn.on_click(lambda b: generate(preview=False))
    preview_btn.on_click(lambda b: generate(preview=True))

    # ========== LAYOUT ==========
    ui = widgets.VBox([
        header,

        # Preset Section
        widgets.VBox([
            preset_label,
            widgets.HBox([preset_category_dropdown, preset_dropdown]),
            preset_info,
        ], layout=widgets.Layout(
            padding='15px',
            border='1px solid #e0e0e0',
            border_radius='12px',
            margin='0 0 16px 0'
        )),

        # Voice Section
        widgets.VBox([
            voice_label,
            voice_mode,
            voice_container,
        ], layout=widgets.Layout(
            padding='15px',
            border='1px solid #e0e0e0',
            border_radius='12px',
            margin='0 0 16px 0'
        )),

        # Text Section
        widgets.VBox([
            text_label,
            text_input,
            char_count,
        ], layout=widgets.Layout(
            padding='15px',
            border='1px solid #e0e0e0',
            border_radius='12px',
            margin='0 0 16px 0'
        )),

        # Settings Section
        widgets.VBox([
            settings_label,
            use_custom_settings,
            settings_container,
            widgets.HTML('<hr style="margin: 12px 0; border: none; border-top: 1px solid #eee;">'),
            effects_label,
            widgets.HBox([effect_normalize, effect_trim, effect_fade]),
        ], layout=widgets.Layout(
            padding='15px',
            border='1px solid #e0e0e0',
            border_radius='12px',
            margin='0 0 16px 0'
        )),

        # Generate
        generate_btn,
        preview_btn,
        status,
        output,

    ], layout=widgets.Layout(max_width='900px', padding='10px'))

    return ui


# ========== TAB: MULTI-VOICE ==========
def build_multivoice_ui():
    header = widgets.HTML("""
    <div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); color: white; padding: 20px; border-radius: 12px; margin-bottom: 16px;">
        <h2 style="margin: 0;">🎭 Multi-Voice Conversation</h2>
        <p style="margin: 5px 0 0; opacity: 0.9;">Create dialogues with multiple voices</p>
    </div>
    """)

    # Voice shortcuts reference
    shortcuts = widgets.HTML("""
    <div style="background: #f5f7ff; padding: 12px; border-radius: 8px; margin-bottom: 12px;">
        <b>Voice Shortcuts:</b><br>
        <code>Jenny</code>, <code>Guy</code>, <code>Aria</code>, <code>Davis</code>, <code>Sonia</code>, <code>Ryan</code>, <code>Natasha</code>, <code>William</code>
    </div>
    """)

    text_area = widgets.Textarea(
        value="""Jenny: Welcome to today's podcast! We have a special guest.
Guy: Thanks for having me, Jenny. I'm excited to be here.
Jenny: So tell us, what inspired you to start this project?
Guy: Well, it all began when I realized there was a gap in the market.
Jenny: That's fascinating! Let's dive deeper into that.""",
        layout=widgets.Layout(width='100%', height='200px')
    )

    gap_slider = widgets.IntSlider(
        value=400, min=100, max=1500, step=50,
        description='Gap (ms):',
        layout=widgets.Layout(width='50%')
    )

    generate_btn = widgets.Button(
        description='🎭 Generate Conversation',
        button_style='success',
        layout=widgets.Layout(width='100%', height='50px')
    )

    output = widgets.Output()

    voice_shortcuts = {
        'jenny': 'en-US-JennyNeural',
        'guy': 'en-US-GuyNeural',
        'aria': 'en-US-AriaNeural',
        'davis': 'en-US-DavisNeural',
        'sonia': 'en-GB-SoniaNeural',
        'ryan': 'en-GB-RyanNeural',
        'natasha': 'en-AU-NatashaNeural',
        'william': 'en-AU-WilliamNeural',
    }

    def on_generate(b):
        with output:
            clear_output()
            print("🎭 Parsing conversation...")

            lines = [l.strip() for l in text_area.value.split('\n') if l.strip()]
            segments = []

            for line in lines:
                if ':' in line:
                    voice_name, text = line.split(':', 1)
                    voice_name = voice_name.strip().lower()
                    voice_id = voice_shortcuts.get(voice_name, 'en-US-JennyNeural')
                    segments.append({'voice': voice_id, 'text': text.strip()})

            if not segments:
                print("❌ No valid segments found!")
                return

            print(f"📝 Found {len(segments)} segments")
            print("⏳ Generating...")

            try:
                result = tts.synthesize_multivoice(segments, gap_ms=gap_slider.value)
                print(f"\n✅ Generated! Duration: {result['duration']:.1f}s")
                display(Audio(result['path'], autoplay=True))
            except Exception as e:
                print(f"❌ Error: {e}")

    generate_btn.on_click(on_generate)

    return widgets.VBox([
        header,
        shortcuts,
        text_area,
        gap_slider,
        generate_btn,
        output
    ])


# ========== TAB: BATCH ==========
def build_batch_ui():
    header = widgets.HTML("""
    <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); color: white; padding: 20px; border-radius: 12px; margin-bottom: 16px;">
        <h2 style="margin: 0;">📦 Batch Generation</h2>
        <p style="margin: 5px 0 0; opacity: 0.9;">Generate multiple audio files at once</p>
    </div>
    """)

    text_area = widgets.Textarea(
        value="""Welcome to our service.
Thank you for your purchase.
Your order has been confirmed.
Please contact support if you need help.
Have a great day!""",
        placeholder='One text per line...',
        layout=widgets.Layout(width='100%', height='180px')
    )

    preset_dropdown = widgets.Dropdown(
        options=[(v['name'], k) for k, v in CONTENT_PRESETS.items()],
        value='corporate',
        description='Preset:',
        layout=widgets.Layout(width='300px')
    )

    merge_check = widgets.Checkbox(value=False, description='Merge into single file')
    gap_slider = widgets.IntSlider(value=500, min=0, max=2000, step=100, description='Gap (ms):')

    progress = widgets.IntProgress(value=0, min=0, max=100, description='Progress:')

    generate_btn = widgets.Button(
        description='🚀 Generate All',
        button_style='success',
        layout=widgets.Layout(width='100%', height='50px')
    )

    output = widgets.Output()

    def on_generate(b):
        with output:
            clear_output()

            lines = [l.strip() for l in text_area.value.split('\n') if l.strip()]
            if not lines:
                print("❌ No text found!")
                return

            progress.max = len(lines)
            progress.value = 0

            preset = CONTENT_PRESETS.get(preset_dropdown.value)
            outputs = []

            for i, line in enumerate(lines):
                print(f"[{i+1}/{len(lines)}] Generating: {line[:40]}...")

                result = tts.synthesize(
                    text=line,
                    voice=preset['voice'],
                    speed=preset['speed'],
                    pitch=preset['pitch']
                )
                outputs.append(result['path'])
                progress.value = i + 1

            print(f"\n✅ Generated {len(outputs)} files!")

            if merge_check.value and len(outputs) > 1:
                print("🔗 Merging files...")
                segments = [tts.audio_processor.load(f) for f in outputs]
                merged = tts.audio_processor.concatenate(segments, gap_slider.value)
                merged_path = str(DIRS['audio'] / f"batch_merged_{int(time.time())}.mp3")
                tts.audio_processor.save(merged, merged_path)
                print(f"✅ Merged file: {merged_path}")
                display(Audio(merged_path, autoplay=True))
            else:
                for path in outputs[-3:]:  # Show last 3
                    display(Audio(path))

    generate_btn.on_click(on_generate)

    return widgets.VBox([
        header,
        text_area,
        widgets.HBox([preset_dropdown]),
        widgets.HBox([merge_check, gap_slider]),
        generate_btn,
        progress,
        output
    ])


# ========== TAB: VOICE LIBRARY ==========
def build_voice_library():
    header = widgets.HTML("""
    <div style="background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); color: #333; padding: 20px; border-radius: 12px; margin-bottom: 16px;">
        <h2 style="margin: 0;">📚 Voice Library</h2>
        <p style="margin: 5px 0 0; opacity: 0.8;">Explore and test all available voices</p>
    </div>
    """)

    # Filters
    lang_options = [('🌍 All Languages', 'all')]
    lang_counts = {}
    for v in ALL_VOICES:
        lang = v['Locale'].split('-')[0]
        lang_counts[lang] = lang_counts.get(lang, 0) + 1
    for lang, count in sorted(lang_counts.items()):
        lang_options.append((f"{lang.upper()} ({count})", lang))

    lang_filter = widgets.Dropdown(options=lang_options, value='all', description='Language:')
    gender_filter = widgets.Dropdown(
        options=[('All', 'all'), ('👩 Female', 'Female'), ('👨 Male', 'Male')],
        value='all',
        description='Gender:'
    )
    search_box = widgets.Text(placeholder='Search voice name...', description='Search:')

    test_text = widgets.Text(
        value='Hello, this is a voice test.',
        description='Test text:',
        layout=widgets.Layout(width='100%')
    )

    filter_btn = widgets.Button(description='🔍 Search', button_style='info')
    output = widgets.Output()

    def do_filter(b):
        with output:
            clear_output()

            voices = ALL_VOICES

            if lang_filter.value != 'all':
                voices = [v for v in voices if v['Locale'].startswith(lang_filter.value)]
            if gender_filter.value != 'all':
                voices = [v for v in voices if v['Gender'] == gender_filter.value]
            if search_box.value:
                voices = [v for v in voices if search_box.value.lower() in v['ShortName'].lower()]

            print(f"📊 Found {len(voices)} voices\n")

            for v in voices[:30]:
                emoji = "👩" if v['Gender'] == 'Female' else "👨"
                name = v['ShortName'].split('-')[-1].replace('Neural', '')

                btn = widgets.Button(
                    description=f"{emoji} {name} ({v['Locale']})",
                    layout=widgets.Layout(width='200px', margin='3px')
                )

                def make_handler(voice_id):
                    def handler(b):
                        result = tts.synthesize(test_text.value, voice=voice_id)
                        display(Audio(result['path'], autoplay=True))
                    return handler

                btn.on_click(make_handler(v['ShortName']))
                display(btn)

            if len(voices) > 30:
                print(f"\n... and {len(voices) - 30} more")

    filter_btn.on_click(do_filter)

    return widgets.VBox([
        header,
        widgets.HBox([lang_filter, gender_filter, search_box, filter_btn]),
        test_text,
        output
    ])


# ========== MAIN TABS ==========
main_ui = build_ui()
multivoice_ui = build_multivoice_ui()
batch_ui = build_batch_ui()
voice_lib_ui = build_voice_library()

tabs = widgets.Tab(children=[main_ui, multivoice_ui, batch_ui, voice_lib_ui])
tabs.set_title(0, '🎙️ Generate')
tabs.set_title(1, '🎭 Multi-Voice')
tabs.set_title(2, '📦 Batch')
tabs.set_title(3, '📚 Voice Library')

display(tabs)

print("\n✅ Pro TTS Studio is ready!")

In [ ]:
#@title 🚀 **CELL 4: Quick API Functions** (Optional)
#@markdown ### Easy-to-use functions for scripting

def speak(text, preset="youtube_explainer", voice=None):
    """Quick synthesis with preset"""
    result = tts.synthesize(text, preset=preset, voice=voice)
    display(Audio(result['path'], autoplay=True))
    return result['path']

def speak_fast(text, voice="en-US-JennyNeural"):
    """Quick fast speech"""
    result = tts.synthesize(text, voice=voice, speed=1.2)
    display(Audio(result['path'], autoplay=True))
    return result['path']

def speak_slow(text, voice="en-US-JennyNeural"):
    """Quick slow speech"""
    result = tts.synthesize(text, voice=voice, speed=0.8)
    display(Audio(result['path'], autoplay=True))
    return result['path']

def speak_podcast(text):
    """Quick podcast style"""
    return speak(text, preset="podcast")

def speak_documentary(text):
    """Quick documentary style"""
    return speak(text, preset="documentary")

def speak_commercial(text):
    """Quick commercial style"""
    return speak(text, preset="commercial_tv")

def speak_meditation(text):
    """Quick meditation style"""
    return speak(text, preset="meditation")

def conversation(script):
    """
    Quick conversation generation

    Example:
        conversation([
            ("Jenny", "Hello!"),
            ("Guy", "Hi there!"),
        ])
    """
    voice_map = {
        'jenny': 'en-US-JennyNeural',
        'guy': 'en-US-GuyNeural',
        'aria': 'en-US-AriaNeural',
        'davis': 'en-US-DavisNeural',
    }

    segments = []
    for voice_name, text in script:
        voice_id = voice_map.get(voice_name.lower(), 'en-US-JennyNeural')
        segments.append({'voice': voice_id, 'text': text})

    result = tts.synthesize_multivoice(segments)
    display(Audio(result['path'], autoplay=True))
    return result['path']

print("""
╔═══════════════════════════════════════════════════════════╗
║                   🚀 QUICK API READY                       ║
╠═══════════════════════════════════════════════════════════╣
║                                                           ║
║  speak("Hello world")           # Default preset          ║
║  speak("Text", preset="podcast")  # With preset           ║
║  speak_fast("Quick message")    # Fast speech             ║
║  speak_slow("Slow message")     # Slow speech             ║
║  speak_podcast("My podcast")    # Podcast style           ║
║  speak_documentary("Nature...") # Documentary style       ║
║  speak_commercial("Buy now!")   # Commercial style        ║
║  speak_meditation("Breathe...") # Meditation style        ║
║                                                           ║
║  conversation([                                           ║
║      ("Jenny", "Hello!"),                                 ║
║      ("Guy", "Hi there!"),                                ║
║  ])                                                       ║
║                                                           ║
╚═══════════════════════════════════════════════════════════╝
""")